#### 词袋模型
   前面几篇笔记几乎都采用的词袋模型，该模型使用one-hot将词转换成向量的形式，故有N个词就会构成N个维度。one-hot向量比较简单也容易理解，但是有很多问题。比如当加入新词时，整个向量的长度会改变，并且存在维度过高难以计算的问题，以及向量的表示方法很难体现两个词之间的关系。
> 词袋模型的特点：离散、高维、稀疏
>> 优点：简单易懂、稀疏存储 
>> 缺点：维度灾难、词汇鸿沟
#### word2vec
    word2vec得到的词的向量形式（简称“词向量”，更学术化的翻译是“词嵌入”）则可以自由控制维度，一般是100左右。
    
    word2vec作为神经概率语言模型的输入，其本身其实是神经概率模型的副产品，是为了通过神经网络学习某个语言模型而产生的中间结果。具体来说，“某个语言模型”指的是“CBOW”和“Skip-gram”。具体学习过程会用到两个降低复杂度的近似方法——Hierarchical Softmax或Negative Sampling。两个模型乘以两种方法，一共有四种实现。
    
   >“CBOW”和“Skip-gram”都是浅层神经网络(不含隐藏层)。
   >> CBOW是通过当前词的上下文关联词预测当前词的概率；
   >> Skip-gram是通过当前词预测上下文关联词的概率
   
   >> Hierarchical Softmax的负例是二叉树的其他路径；
   >> Negative Sampling的负例是随机挑选出来的，随机挑选能提高运行速度，改善模型质量
    
> word2vec的特点：连续、低维、稠密

[参考文档](http://www.hankcs.com/nlp/word2vec.html)

https://spaces.ac.cn/archives/4304

https://blog.csdn.net/u013421629/article/details/82462606

#### gensim.word2vec

In [33]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd
import jieba

In [6]:
# sens = LineSentence('./cnews/cnews.val.txt')

In [24]:
data = pd.read_csv('./cnews/cnews.val.txt', sep='\t', encoding='utf-8', header=None, names=['label','content'])
data.head(1)

,label,content
0,体育,黄蜂vs湖人首发：科比带伤战保罗 加索尔救赎之战 新浪体育讯北京时间4月27日，NBA季后赛...


In [132]:
#停用词
stop_words = pd.read_csv('./cnews/stopwords.txt', index_col = False, quoting=3, sep='\t',
                        names = ['stopword'], encoding='utf-8')

content = data.content.values.tolist()
stopword = stop_words.stopword.values.tolist()

texts = []

for i in content[:500]: 
    #均为体育新闻
    seg = list(j for j in jieba.lcut(i) if str(j) not in stopword and len(j)>1)
    texts.append(seg)

In [133]:
print(texts[:2])

[['黄蜂', '湖人', '首发', '科比', '带伤', '保罗', '加索尔', '救赎', '之战', '新浪', '体育讯', '北京', '时间', 'NBA', '季后赛', '首轮', '洛杉矶', '湖人', '主场', '迎战', '新奥尔良', '黄蜂', '此前', '比赛', '战成', '本场', '比赛', '两支', '球队', '赛前', '公布', '首发', '阵容', '湖人队', '费舍尔', '科比', '阿泰斯特', '加索尔', '拜纳姆', '黄蜂队', '保罗', '贝里', '内利', '阿里', '兰德', '奥卡福', '新浪', 'NBA', '官方', '微博', '新浪', 'NBA', '湖人', '新闻动态', '微博', '新浪', 'NBA', '专题', '黄蜂', '湖人', '图文', '直播室', '新浪', '体育'], ['1.7', '击救', '马刺', '王朝', '危难', '新秀', '新浪', '体育讯', '刚刚', '结束', '比赛', '回到', '主场', '马刺', '加时', '110', '103', '惊险', '战胜', '主场', '观众', '见证', '黑八', '尴尬', '常规', '时间', '关头', '加里', '尼尔', '命中', '一记', '三分', '进球', '马刺', '比赛', '带进', '加时', '最终', '翻盘', '成功', '波波维奇', '教练', '安排', '详细', '告诉', '机会', '出手', '邓肯', '掩护', '挡住', '防守', '球员', '投篮', '视野', '出手', '命中', '这记', '价值连城', '尼尔', '依然', '低调', '问及', '职业生涯', '投篮', '没错', '目前为止', '赛季', '听说', '尼尔', '西部', '第一', '马刺', '缺少', '拼图', '季后赛', '赢球', '回家', '钓鱼', '关键', '比赛', '命中', '球队', '续命', '关键球', '成功', '背后', '隐藏', '一段', '鲜为人知', '艰难', '历程', '效力', '拉塞尔', '大学', '拿下', '赛区'

In [134]:
sens = texts.copy()

In [135]:
#训练模型
model = Word2Vec(sens, size=128, window=3, min_count=5, workers=8)

In [136]:
# 保存模型
model.save('./word_emdedding')

In [137]:
#加载模型
model = Word2Vec.load('./word_emdedding')

In [138]:
#测试词语之间的相似度
items = model.most_similar('体育')
for i,j in items:
    print(i,j)

黎双富 0.9984898567199707
迈阿密 0.9984685778617859
官方 0.9976723194122314
雅虎 0.9970170259475708
NBA 0.9964408874511719
微博 0.9938219785690308
发自 0.9937679171562195
00 0.9932857751846313
新闻动态 0.9931809306144714
官网 0.9911702871322632


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [139]:
#测试词语之间的相似度
items = model.wv.most_similar('体育')
for i,j in items:
    print(i,j)

黎双富 0.9984898567199707
迈阿密 0.9984685778617859
官方 0.9976723194122314
雅虎 0.9970170259475708
NBA 0.9964408874511719
微博 0.9938219785690308
发自 0.9937679171562195
00 0.9932857751846313
新闻动态 0.9931809306144714
官网 0.9911702871322632


In [140]:
tt=model.similarity('得分',  '投中')

print(tt)

0.9967143040273748


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [141]:
tt=model.wv.similarity('能力',  '成功')

print(tt)

0.9985934569655914


In [142]:
tt=model.wv.similarity('新浪',  '成功')

print(tt)

0.6195670108377953


In [143]:
tt=model.wv.similarity('成功',  '失败')

print(tt)

0.9985490506450232


In [144]:
tt=model.wv.similarity('成功',  '尴尬')

print(tt)

0.9990024484055975


In [131]:
tt=model.wv.similarity('失败',  '尴尬')

print(tt)

0.9810751637767183


以上文本全部是体育新闻，相似度过高，所以训练效果不够好，可以增加训练集或者加入其他类型的文本进行训练